In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from sklearn.datasets import load_boston
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import ShuffleSplit

import pymc3 as pm

from modules.neural_networks import BayesianMLP

In [ ]:
X, y = load_boston(return_X_y=True)

for tr_i, ts_i in ShuffleSplit(n_splits=1).split(X, y):
    
    X_tr, X_ts = X[tr_i], X[ts_i]
    y_tr, y_ts = y[tr_i], y[ts_i]
    
    scaler = MinMaxScaler()
    scaler.fit(X_tr)
    X_tr = scaler.transform(X_tr)
    X_ts = scaler.transform(X_ts)

# Gaussian Likelyhood

In [ ]:
gaussian_perceptron = BayesianMLP(
    X=X_tr, 
    y=y_tr, 
    shape_out=1, 
    likelyhood_model='gaussian_lk',
    layers=(32, 16, 8, 4), 
    activation='relu',
    batch_size=16,
    prior=pm.Normal,
    mu=0,
    sigma=1
)
gaussian_perceptron.show_graph()

In [ ]:
gaussian_perceptron.fit(n=100000)
gaussian_traces = gaussian_perceptron.predict(
    X_ts, 
    y_ts, 
    ['y']
)

# Student-t Likelyhood

In [ ]:
student_perceptron = BayesianMLP(
    X=X_tr, 
    y=y_tr, 
    shape_out=1, 
    likelyhood_model='student_lk',
    layers=(32, 16, 8, 4), 
    activation='relu',
    batch_size=16,
    prior=pm.Normal,
    mu=0,
    sigma=1
)
student_perceptron.show_graph()

In [ ]:
student_perceptron.fit(n=100000)
student_traces = student_perceptron.predict(
    X_ts, 
    y_ts, 
    ['y']
)